In [ ]:
# Imports
import os
import shutil
import tensorflow as tf

import paths

from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2

from google.protobuf import text_format

## Training Setup

In [ ]:
# Labels that our model will predict
labels = [
    {
        'name':'K', 
        'id':1
    },
    {
        'name':'O', 
        'id':2
    },
    {
        'name':'V', 
        'id':3
    },
]

with open(paths.ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
# Creating TF records for our model to be able to train
!python {paths.SCRIPTS_PATH + '/generate_tfrecord.py'} -x {paths.IMAGE_PATH + '/train'} -l {paths.ANNOTATION_PATH + '/label_map.pbtxt'} -o {paths.ANNOTATION_PATH + '/train.record'}
!python {paths.SCRIPTS_PATH + '/generate_tfrecord.py'} -x{paths.IMAGE_PATH + '/test'} -l {paths.ANNOTATION_PATH + '/label_map.pbtxt'} -o {paths.ANNOTATION_PATH + '/test.record'}

In [ ]:
# Create the destination directory if it does not exist
dest_dir = os.path.join(paths.MODEL_PATH, paths.CUSTOM_MODEL_NAME)
if not os.path.exists(dest_dir):
  os.makedirs(dest_dir)
  print(f"Directory created: {dest_dir}")
else:
  print(f"Directory already exists: {dest_dir}")

# Define the source and destination paths for the config file
source_config = os.path.join(paths.PRETRAINED_MODEL_PATH,
  'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
  'pipeline.config')
dest_config = os.path.join(dest_dir, 'pipeline.config')

# Copy config from source to destination
try:
  shutil.copy(source_config, dest_config)
  print(f"Copied '{source_config}' to '{dest_config}'")
except Exception as e:
  print(f"Error copying file: {e}")

In [ ]:
config = config_util.get_configs_from_pipeline_file(paths.CONFIG_PATH)
print(config)

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(paths.CONFIG_PATH, "r") as f:
  proto_str = f.read()
  text_format.Merge(proto_str, pipeline_config)

In [ ]:
# Setup config for the model
pipeline_config.model.ssd.num_classes = 3
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = paths.PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= paths.ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [paths.ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = paths.ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [paths.ANNOTATION_PATH + '/test.record']

In [ ]:
# Save new config
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(paths.CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

## Get training command

In [ ]:
# Get command for model training
print(f"python {paths.APIMODEL_PATH}/research/object_detection/model_main_tf2.py --model_dir={paths.MODEL_PATH}/{paths.CUSTOM_MODEL_NAME} --pipeline_config_path={paths.MODEL_PATH}/{paths.CUSTOM_MODEL_NAME}/pipeline.config --num_train_steps=100000")

## Get evaluation command

In [ ]:
print(f"python {paths.APIMODEL_PATH}/research/object_detection/model_main_tf2.py --model_dir={paths.MODEL_PATH}/{paths.CUSTOM_MODEL_NAME} --pipeline_config_path={paths.MODEL_PATH}/{paths.CUSTOM_MODEL_NAME}/pipeline.config --checkpoint_dir={paths.MODEL_PATH}/{paths.CUSTOM_MODEL_NAME}")

# --model_dir + train ili eval folder u tom folderu mogu runnati tensorboard --logdir=. i dobiti performanse modela